**Przegląd deskryptorów**
Deskryptory zostały wprowadzone do języka Python w wersji 2.2. Umożliwiają deweloperom dodawanie zarządzanych atrybutów do obiektów. Metody potrzebne do utworzenia deskryptora to __get__, __set__ i __delete__. Zdefiniowanie dowolnej z tych metod oznacza utworzenie deskryptora.

Idea deskryptora polega na pobieraniu, ustawianiu lub usuwaniu atrybutów ze słownika naszego obiektu. Kiedy uzyskujemy dostęp do atrybutu klasy, rozpoczyna się łańcuch wyszukiwania. Jeśli uzyskana wartość jest obiektem z jedną ze zdefiniowanych metod deskryptora, to ta metoda deskryptora zostanie wywołana.

Deskryptory odpowiadają za wiele magicznych aspektów wewnętrznego działania Pythona. To dzięki nim działają właściwości, metody, a nawet funkcja super. Deskryptory są również wykorzystywane do implementacji klas w nowym stylu, które również zostały wprowadzone w Pythonie 2.2.

**Protokół deskryptorów w Pythonie**
Protokół tworzenia deskryptora jest naprawdę prosty. Wystarczy zdefiniować jedną lub więcej z następujących metod:
- __get__ (self, obj, type=None): zwraca wartość atrybutu
- __set__ (self, obj, value): zwraca None (ustawia wartość atrybutu)
- __delete__ (self, obj): zwraca None (usuwa atrybut)

Zdefiniowanie co najmniej jednej z tych metod oznacza utworzenie deskryptora.

Jeśli uda nam się zdefiniować zarówno __get__, jak i __set__, utworzymy deskryptor danych (data descriptor). Deskryptor posiadający tylko zdefiniowaną metodę __get__ nazywany jest deskryptorem bez danych (non-data descriptor) i zwykle jest używany do metod. Powodem tego rozróżnienia typów deskryptorów jest to, że jeśli słownik instancji posiada deskryptor danych, to deskryptor ten będzie miał pierwszeństwo podczas wyszukiwania. Natomiast jeśli słownik instancji posiada wpis odpowiadający deskryptorowi bez danych, to pierwszeństwo będzie miał wpis słownika, a nie deskryptor.

Możemy również utworzyć deskryptor tylko do odczytu, jeśli zdefiniujemy zarówno __get__, jak i __set__, ale wywołamy wyjątek AttributeError w momencie wywołania metody __set__.

**Jak wywoływane są deskryptory**
Najczęstszym sposobem wywołania deskryptora jest automatyczne jego uruchomienie podczas uzyskiwania dostępu do atrybutu. Typowy przykład to moj_obiekt.nazwa_atrybutu. Spowoduje to, że nasz obiekt będzie szukał nazwa_atrybutu w obiekcie moj_obiekt. Jeśli nazwa_atrybutu posiada zdefiniowaną metodę __get__(), zostanie wywołana nazwa_atrybutu.__get__(moj_obiekt). Wszystko to zależy od tego, czy nasza instancja jest obiektem, czy klasą.

Magia za tym kryje się w magicznej metodzie __getattribute__, która zamieni moj_obiekt.a na: type(moj_obiekt).__dict__['a'].__get__(a, type(a)). Więcej na temat implementacji można przeczytać w dokumentacji języka Python.

Zgodnie z dokumentacją, należy pamiętać o kilku kwestiach dotyczących wywoływania deskryptorów:
- Deskryptor jest wywoływany za pomocą domyślnej implementacji metody __getattribute__.
- Jeśli przesłonimy __getattribute__, uniemożliwi to automatyczne wywołanie deskryptora.
- object.__getattribute__() i type.__getattribute__() nie wywołują __get__() w ten sam sposób.
- Deskryptor danych zawsze, ZAWSZE będzie miał pierwszeństwo przed słownikiem instancji.
- Deskryptor bez danych może zostać nadpisany przez słownik instancji.

**Przykłady Deskryptorów**
W tym momencie możemy być zdezorientowani, do czego w ogóle służą deskryptory. Podczas nauki nowych zagadnień zawsze przydatne są przykłady, które pokazują działanie w praktyce. Dlatego w tej lekcji przyjrzymy się kilku przykładom, abyśmy wiedzieli, jak wykorzystywać deskryptory we własnym kodzie!

**Prosty przykład deskryptora danych**
Zacznijmy od napisania naprawdę prostego deskryptora danych, a następnie wykorzystania go w klasie. Poniższy przykład opiera się na dokumentacji języka Python:

In [1]:
class MyDescriptor():
    """
    A simple demo descriptor
    """
    def __init__(self, initial_value=None, name='my_var'):
        self.var_name = name
        self.value = initial_value

    def __get__(self, obj, objtype):
        print('Getting', self.var_name)
        return self.value

    def __set__(self, obj, value):
        msg = 'Setting {name} to {value}'
        print(msg.format(name=self.var_name, value=value))
        self.value = value

class MyClass():
    desc = MyDescriptor(initial_value='Mike', name='desc')
    normal = 10

if __name__ == '__main__':
    c = MyClass()
    print(c.desc)
    print(c.normal)
    c.desc = 100
    print(c.desc)

Getting desc
Mike
10
Setting desc to 100
Getting desc
100


Tutaj tworzymy klasę i definiujemy trzy magiczne metody:
- __init__: nasz konstruktor, który przyjmuje wartość i nazwę naszej zmiennej (linie 5-7)
- __get__: wypisuje bieżącą nazwę zmiennej i zwraca wartość (linie 9-11)
- __set__: wypisuje nazwę naszej zmiennej i wartość, którą właśnie przypisaliśmy, a następnie ustawia samą wartość (linie 13-16)

Następnie tworzymy klasę, która tworzy instancję naszego deskryptora jako atrybut klasy, a także tworzy zwykły atrybut klasy (linie 18-20). Następnie uruchamiamy kilka "testów", tworząc instancję naszej normalnej klasy i uzyskując dostęp do atrybutów klasy.

Jak widać, gdy uzyskujemy dostęp do c.desc, wypisuje się komunikat "Pobieranie" i drukujemy to, co zwraca, czyli "Mike". Następnie drukujemy wartość zwykłego atrybutu klasy. Na koniec zmieniamy wartość zmiennej deskryptora, co powoduje wydrukowanie komunikatu "Ustawianie". Sprawdzamy również dwukrotnie bieżącą wartość, aby upewnić się, że została faktycznie ustawiona, dlatego widzimy ten ostatni komunikat "Pobieranie".

Python używa deskryptorów wewnętrznie do tworzenia właściwości, metod związanych/niezwiązanych i metod klasowych. Jeśli zajrzymy do dokumentacji klasy property w dokumentacji języka Python, zobaczymy, że ściśle przestrzega ona protokołu deskryptorów:

```
property(fget=None, fset=None, fdel=None, doc=None)
```

Powyższe wyraźnie pokazuje, że klasa property posiada metodę pobierającą, ustawiającą i usuwającą.

**Walidacja przy użyciu deskryptora**
Spójrzmy na kolejny przykład, w którym wykorzystamy deskryptor do walidacji:

In [2]:
from weakref import WeakKeyDictionary

class Drinker:
    def __init__(self):
        self.req_age = 21
        self.age = WeakKeyDictionary()

    def __get__(self, instance_obj, objtype):
        return self.age.get(instance_obj, self.req_age)

    def __set__(self, instance, new_age):
        if new_age < 21:
            msg = '{name} is too young to legally imbibe'
            raise Exception(msg.format(name=instance.name))
        self.age[instance] = new_age
        print('{name} can legally drink in the USA'.format(
            name=instance.name))

    def __delete__(self, instance):
        del self.age[instance]


class Person:
    drinker_age = Drinker()

    def __init__(self, name, age):
        self.name = name
        self.drinker_age = age


p = Person('Miguel', 30)
p = Person('Niki', 13)

Miguel can legally drink in the USA


Exception: Niki is too young to legally imbibe

Tworzymy ponownie klasę deskryptora. W tym przypadku korzystamy z biblioteki weakref Pythona, a konkretnie z klasy WeakKeyDictionary. Jest to sprytna klasa, która tworzy słownik mapujący klucze słabo (weakly). Oznacza to, że gdy w słowniku nie ma silnych odwołań do klucza, to klucz i jego wartość zostaną odrzucone. W tym przykładzie wykorzystujemy to, aby zapobiec wiszeniu w pamięci instancji klasy Person w nieskończoność.

Jednak najważniejsza część deskryptora znajduje się w metodzie __set__. Tutaj sprawdzamy, czy parametr wiek instancji jest większy niż 21 lat, co jest wymagane w USA do spożywania napojów alkoholowych. Jeśli nasz wiek jest niższy, zostanie zgłoszony wyjątek. W przeciwnym razie zostanie wydrukowane imię osoby i komunikat. Aby przetestować nasz deskryptor, tworzymy dwie instancje, jedną z wiekiem powyżej 21 lat, a drugą z wiekiem poniżej.

Oczywiście wszystko działało tak, jak powinno, ale nie jest do końca jasne, jak to działa. Dzieje się tak, ponieważ gdy ustawiamy wartość drinker_age, Python zauważa, że jest to deskryptor. Python wie, że drinker_age jest deskryptorem, ponieważ zdefiniowaliśmy go jako atrybut klasy:

```
drinker_age = Drinker()
```

Tak więc, gdy ustawiamy wartość, wywołujemy faktycznie metodę __set__ naszego deskryptora, która przekazuje instancję i wiek, który próbujemy ustawić. Jeśli wiek jest mniejszy niż 21, zgłaszamy wyjątek z własną wiadomością. W przeciwnym razie wypisujemy komunikat, że jesteśmy wystarczająco dorośli.

Wracając do tego, jak to wszystko działa, jeśli spróbujemy wydrukować drinker_age, Python wykona Person.drinker_age.__get__. Ponieważ drinker_age jest deskryptorem, to jego metoda __get__ jest faktycznie wywoływana.

Jeśli chcielibyśmy ustawić wartość drinker_age, zrobilibyśmy to tak:

```
p.drinker_age = 32
```

Wtedy Python wywołałby Person.drinker_age.__set__, a ponieważ ta metoda jest również zaimplementowana w naszym deskryptorze, to właśnie metoda deskryptora zostanie wywołana. Po kilkukrotnym prześledzeniu kodu szybko zrozumiemy, jak to wszystko działa.

Najważniejszą rzeczą do zapamiętania jest to, że deskryptory są powiązane z klasami, a nie z instancjami.